C'est normal d'avoir des difficultés avec le code, surtout sur un projet qui combine scraping, API et calculs géométriques. Pour la soutenance, ce qui compte, c'est de montrer que **vous avez compris la logique** de ce que vous faites, même si le code n'est pas parfait.

Voici un plan d'action en **3 étapes clés**, avec le code prêt à l'emploi et, pour chaque bloc, l'argumentaire "Spécial Soutenance".

---

###Étape 1 : Récupérer les données propres des gares (Open Data)Au lieu de scraper des coordonnées approximatives, on télécharge le fichier officiel de la SNCF.

**Le Code (à copier dans un nouveau notebook ou à la suite de `main.ipynb`) :**

```python
import pandas as pd

# 1. URL stable de l'API SNCF Open Data (export CSV)
url_sncf = "https://ressources.data.sncf.com/api/explore/v2.1/catalog/datasets/liste-des-gares/exports/csv"

# 2. Chargement des données
print("Téléchargement des données SNCF en cours...")
df_sncf = pd.read_csv(url_sncf, sep=";")

# 3. Extraction et nettoyage des coordonnées
# On ne garde que le nom et la position. Le format est souvent "Lat, Lon" dans une seule colonne.
df_gares_final = df_sncf[['libelle', 'geo_point_2d']].dropna().copy()

# 4. Séparation de la colonne unique "48.85, 2.35" en deux colonnes numériques
df_gares_final[['Lat', 'Lon']] = df_gares_final['geo_point_2d'].str.split(',', expand=True).astype(float)

# 5. Renommer pour être clair
df_gares_final = df_gares_final.rename(columns={'libelle': 'Nom_Gare'})

print(f"Nombre de gares récupérées : {len(df_gares_final)}")
df_gares_final.head()

```

####🎓 L'Explication pour la Soutenance (Ligne par Ligne)* **Ligne 1 (`url_sncf = ...`)** : *"Nous avons choisi d'utiliser l'API Open Data de la SNCF plutôt que le scraping pour les coordonnées géographiques."*
* **Justification :** C'est une source **officielle** (fiabilité) et **stable** (le code ne cassera pas si Wikipédia change son design).


* **Ligne 2 (`pd.read_csv(..., sep=";")`)** : *"On charge directement le CSV depuis l'URL."*
* **Justification :** Le séparateur est `;` car les CSV français utilisent souvent la virgule pour les décimales.


* **Ligne 3 (`dropna()`)** : *"On supprime les lignes vides."*
* **Justification :** Une gare sans coordonnées GPS est inutile pour notre modèle, on nettoie les données dès l'entrée.


* **Ligne 4 (`str.split`)** : *"On sépare la chaîne de caractères géographique en deux nombres flottants (latitude et longitude)."*
* **Justification :** Pour faire des calculs mathématiques de distance, l'ordinateur a besoin de deux nombres distincts, pas d'une phrase textuelle.



---

###Étape 2 : Le cœur du projet - Calculer la distance (BallTree)C'est l'étape la plus technique. Vous avez 300 000 transactions immobilières et 2 000 gares.
Si vous faites une boucle simple (`pour chaque maison, regarde toutes les gares`), l'ordinateur va faire 300 000 x 2 000 = **600 millions d'opérations**. Ça prendra des heures. On utilise donc un algorithme optimisé : le **BallTree**.

**Le Code :**

```python
from sklearn.neighbors import BallTree
import numpy as np

# Supposons que 'df2' est votre DataFrame immobilier nettoyé contenant 'lat' et 'lon'
# et 'df_gares_final' est celui de l'étape 1

# 1. Conversion des coordonnées en radians
# La Terre est une sphère, les maths marchent en radians, pas en degrés.
gps_gares = np.radians(df_gares_final[['Lat', 'Lon']].values)
gps_immo = np.radians(df2[['lat', 'lon']].values) # Assurez-vous d'avoir ces colonnes dans df2

# 2. Construction de l'arbre (Indexation spatiale)
tree = BallTree(gps_gares, metric='haversine')

# 3. Requête pour trouver le plus proche voisin (k=1)
# Cela renvoie la distance et l'index de la gare la plus proche
distances, indices = tree.query(gps_immo, k=1)

# 4. Conversion de la distance (radians -> km)
# Rayon de la Terre approx = 6371 km
df2['distance_gare_km'] = distances * 6371

# 5. Récupération du nom de la gare la plus proche
# On utilise les indices trouvés par l'arbre pour aller chercher le nom dans le dataframe des gares
df2['gare_proche'] = df_gares_final.iloc[indices.flatten()]['Nom_Gare'].values

print(df2[['adresse', 'gare_proche', 'distance_gare_km']].head())

```

####🎓 L'Explication pour la Soutenance* **Ligne 1 (`np.radians`)** : *"Nous convertissons les degrés GPS en radians."*
* **Justification :** Les formules de trigonométrie sphérique (pour calculer une distance sur une sphère comme la Terre) fonctionnent en radians.


* **Ligne 2 (`BallTree`)** : *"Nous construisons une structure de données en arbre, appelée BallTree."*
* **Justification (CLÉ pour la note) :** C'est un choix d'**optimisation algorithmique**. Au lieu de calculer bêtement toutes les distances (complexité O(N*M)), l'arbre permet de diviser l'espace et de trouver le voisin le plus proche très vite (complexité logarithmique O(log N)). C'est ce qui rend le code exécutable sur un gros dataset.


* **Ligne 3 (`metric='haversine'`)** : *"On utilise la formule de Haversine."*
* **Justification :** C'est la formule mathématique qui calcule la distance "à vol d'oiseau" sur une sphère, tenant compte de la courbure de la Terre.


* **Ligne 4 (`k=1`)** : *"On demande seulement le 1er voisin."*
* **Justification :** On ne veut que la gare la plus proche, pas les 10 plus proches.



---

###Étape 3 : La brique "Scraping" (Pour valider le cours)Puisque vous avez "triché" (intelligemment) à l'étape 1 en prenant l'Open Data, il faut montrer au jury que vous savez quand même scraper.
Nous allons scraper **la population des préfectures** sur Wikipédia pour voir si le prix dépend aussi de la taille de la ville (et pas juste du TGV).

**Le Code :**

```python
import requests
from bs4 import BeautifulSoup

def get_population_wikipedia(ville):
    # 1. Construction de l'URL Wikipédia
    # On remplace les espaces par des underscores (ex: "Bourg en Bresse" -> "Bourg_en_Bresse")
    base_url = "https://fr.wikipedia.org/wiki/"
    url = base_url + ville.replace(" ", "_")

    try:
        # 2. Requête HTTP
        response = requests.get(url)
        if response.status_code != 200:
            return None

        # 3. Parsing HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # 4. Recherche de l'Infobox (le tableau récapitulatif à droite sur Wikipédia)
        infobox = soup.find("div", {"class": "infobox_v3"})

        # 5. Recherche de la ligne "Population"
        # C'est du scraping "artisanal" : on cherche le texte qui contient "Population"
        rows = infobox.find_all("tr")
        for row in rows:
            if "Population" in row.text:
                # On nettoie le texte (enlève les espaces insécables, etc)
                return row.text.split("hab.")[0].replace("\xa0", "").replace(" ", "")

    except Exception as e:
        return None

# Test sur une ville
pop = get_population_wikipedia("Lyon")
print(f"Population scrapée : {pop}")

```

####🎓 L'Explication pour la Soutenance* **Ligne 1 (URL)** : *"On dynamise l'URL."*
* **Justification :** Wikipédia a une structure d'URL prévisible, ce qui nous permet d'automatiser la recherche par nom de ville.


* **Ligne 3 (`BeautifulSoup`)** : *"On utilise la librairie BS4 pour transformer le HTML brut en objet navigable."*
* **Ligne 4 (`infobox_v3`)** : *"On cible la classe CSS spécifique."*
* **Justification :** Après avoir inspecté le code source de la page (F12), on a remarqué que les données clés sont toujours dans une `div` appelée `infobox_v3` ou `infobox`. Cela limite la recherche et évite de lire tout le texte de l'article.


* **Gestion des erreurs (`try/except`)** : *"On a ajouté une gestion d'erreurs."*
* **Justification :** En scraping, les pages sont hétérogènes. Si une page n'existe pas ou n'a pas d'infobox, le code ne doit pas planter, il doit juste renvoyer `None` et passer à la suivante.



###Résumé pour votre projet1. Lancez le code de l'**Étape 1** pour avoir `df_gares_final`.
2. Assurez-vous que votre `df2` (immobilier) a bien des colonnes `lat` et `lon`.
* *Attention :* Dans votre `main.ipynb`, vous géocodez ligne par ligne avec une boucle. Pour 300 000 lignes, **ça va planter**.
* *Conseil d'ami :* Pour le projet, travaillez sur un **échantillon** (ex: `df2 = df2.sample(10000)`). Expliquez à la soutenance que vous avez réduit la taille pour des raisons de performance de l'API de géocodage, mais que le code marcherait sur tout le fichier avec plus de temps.


3. Lancez le code de l'**Étape 2** pour ajouter la colonne `distance_gare_km` à votre échantillon.
4. Faites un graphique simple (nuage de points : axe X = distance, axe Y = prix m2).

Est-ce que ces explications vous semblent claires pour les présenter ?